In [1]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, array_to_img
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
from keras import backend as K
import pandas as pd
import numpy as np
from glob import glob

Using TensorFlow backend.


In [10]:
img_width, img_height = 224, 224

train_data_dir = '/home/sage/ml_playground/catsvsdogs/PetImages/train'
validation_data_dir = '/home/sage/ml_playground/catsvsdogs/PetImages/val'

nb_train_samples = !find {train_data_dir} | grep jpg | wc -l
nb_train_samples = int(nb_train_samples[0])

nb_validation_samples = !find {validation_data_dir} | grep jpg | wc -l
nb_validation_samples = int(nb_validation_samples[0])

n_classes = 2

epochs = 2
batch_size = 16

In [3]:
nb_train_samples, nb_validation_samples

(23529, 1240)

In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', name='fc1')(x)

prediction = Dense(n_classes, activation='softmax', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=prediction)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
for layer in model.layers:
    if layer.name in ['predictions']:
        continue
    layer.trainable = False

In [11]:
df = pd.DataFrame(([layer.name, layer.trainable] for layer in model.layers), columns=['layer', 'trainable'])

In [6]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(directory=train_data_dir,
                                                    target_size=[img_width, img_height],
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(directory=validation_data_dir,
                                                              target_size=[img_width, img_height],
                                                              batch_size=batch_size,
                                                              class_mode='categorical')

Found 23529 images belonging to 2 classes.
Found 1240 images belonging to 2 classes.


In [7]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
model.fit_generator(
        train_generator,
        steps_per_epoch= nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
 429/1470 [=======>......................] - ETA: 1253s - loss: 0.3634 - acc: 0.8775

/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 404094976 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 404619264 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/P

 540/1470 [==========>...................] - ETA: 1137s - loss: 0.3455 - acc: 0.8839

/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


1125/1470 [=====================>........] - ETA: 435s - loss: 0.2899 - acc: 0.9031

...

1367/1470 [==========================>...] - ETA: 129s - loss: 0.2807 - acc: 0.9075

/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307363840 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307888128 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 328728576 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385474 bytes but only got 5357. Skipping tag 513
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-pac

1469/1470 [============================>.] - ETA: 1s - loss: 0.2785 - acc: 0.9088

/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
/home/sage/envs/playground/lib/python3.5/site-packages/PIL/TiffImagePlugin.

1470/1470 [==============================] - 1952s - loss: 0.2783 - acc: 0.9089 - val_loss: 0.1206 - val_acc: 0.9732
Epoch 2/2
1470/1470 [==============================] - 1897s - loss: 0.2291 - acc: 0.9293 - val_loss: 0.1188 - val_acc: 0.9649


In [12]:
model.save_weights('/home/sage/ml_playground/catsvsdogs/trained/keras/vgg16_cat_dog_retrained.h5')

model_json_final = model.to_json()

with open("/home/sage/ml_playground/catsvsdogs/trained/keras/vgg16_cat_dog_retrained.json", "w") as json_file:
    json_file.write(model_json_final)